In [1]:
# Import the modules we need
from IPython.core.debugger import set_trace

from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from Crypto.Hash import SHA256

import binascii

from time import time
import random
import  jdc

from collections import OrderedDict
import hashlib
import json

## 1. Wallet

Zusatzinfo: Es ist wichtig die Intuition hinter der asymetrischen Kryptografie zu verstehen. Wer sich darunter nichts vorstellen kann und in der Vorlesung nicht folgen konnte, dem helfen folgende Videos vielleicht weiter:
* [Private Key / Public Key Kryptografie allg.](https://www.youtube.com/watch?v=GSIDS_lvRv4)
* [Eliptic Curve](https://www.youtube.com/watch?v=NF1pwjL9-DE)


Wallets haben zwei wesentliche Funktionen: 
1. Guthabenverwaltung 
2. Transaktionsdurchführung

Die Basis für beide Funktionen stellt der `Private Key` dar. Dieser entspricht einfach einem zufällig erzeugten (sehr langen) Passwort in einem bestimmten Format. Vom `Private Key` kann sowohl der `Public Key` abgeleitet, als auch Transaktionen signiert werden (wird später eingeführt).

In [2]:
class Wallet:
    # Basic Constructor
    def __init__(self):
        keypair = self.generate_keypair()
        self.private_key = keypair['private_key']
        self.public_key = keypair['public_key']

    def generate_keypair(self):
        # TODO:
        # 1. Generate the private key (randomly)
        # 2. Return the private_key and public_key in
        # hexadecimal as dictionary
        private_key = ECC.generate(curve="P-256")
        keypair = {
            "private_key": binascii.hexlify(private_key.export_key(format="DER")).decode("utf-8"),
            "public_key": binascii.hexlify(private_key.public_key().export_key(format="DER")).decode("utf-8")
        }
        return keypair


In [3]:
# Testing it:

wallet_alice = Wallet()
wallet_bob = Wallet()

print("Alice's Wallet:")
print(wallet_alice.private_key)
print('\n')
print(wallet_alice.public_key)
print("Bob's Wallet:")
print(wallet_bob.private_key)
print('\n')
print(wallet_bob.public_key)

Alice's Wallet:
308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042004712fa6588cc024579e6e16cf9610f774d780568c96592f1e3285640a65ed77a14403420004c09adbceec8529afa226268ac405dd378d279d4a96566a39134ebd49e307054332961b1e33567435400aac564d0f73d5290acec333e261019aa0e8a95fa4e8c3


3059301306072a8648ce3d020106082a8648ce3d03010703420004c09adbceec8529afa226268ac405dd378d279d4a96566a39134ebd49e307054332961b1e33567435400aac564d0f73d5290acec333e261019aa0e8a95fa4e8c3
Bob's Wallet:
308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104202b8387c3fb84f74769652de8c1bf71590daab2502d71853fafc087ab79e699e2a14403420004e3ae8418630359efe450cd4dfa7fe0f4da8f960baaa1dddbd176daa19ea5428bd0aeadbce8e0e207909d4708e3c20c794114f4b788c554b3a633c0ae0955dfdb


3059301306072a8648ce3d020106082a8648ce3d03010703420004e3ae8418630359efe450cd4dfa7fe0f4da8f960baaa1dddbd176daa19ea5428bd0aeadbce8e0e207909d4708e3c20c794114f4b788c554b3a633c0ae0955dfdb


#### Wallet zu einem bestimmten Private Key
Wir können die Klasse nun so erweitern, dass wir für den Fall eines schon existierenden `Private Key` ebenfalls eine Wallet erzeugt werden kann. Beachte, dass die `ECC.import_key()` Funktion immer einen binary Input benötigt, wir speichern die Keys aber in der Regel im hexadecimalen Format als String ab. Deshalb is die Konvertierung durch binascii.unhexlify() notwendig. D.h. der `private_key_str` wird bei uns immer im hexadecimalen Format als String dem Wallet Konstruktor übergeben.

In [4]:
class Wallet:
    def __init__(self, private_key_str=None):
        keypair = self.generate_keypair(private_key_str)
        self.private_key = keypair['private_key']
        self.public_key = keypair['public_key']

    @staticmethod
    def generate_keypair(private_key_str):
        # TODO:
        # 1. Copy paste the generate_keypair content from above
        # 2. Extend the function such that whenever the private_key_str is
        # non empty, use the private_key_str to generate the keypair
        # in the required format (same as above). Otherwise just generate 
        # a new private_key.
        if private_key_str is None:
            private_key = ECC.generate(curve="P-256")
        else:
            private_key = ECC.import_key(binascii.unhexlify(private_key_str))

        keypair = {
            "private_key": binascii.hexlify(private_key.export_key(format="DER")).decode("utf-8"),
            "public_key": binascii.hexlify(private_key.public_key().export_key(format="DER")).decode("utf-8")
        }

        return keypair


In [5]:
# Testing it:
# TODO:
# 1. Extract the Alice's private key from her wallet
# 2. Generate a new wallet using Alice's private key and
# 3. Check if the public keys from the new wallet and Alice's
# old wallet are the same.

# 1.
private_key_alice_str = wallet_alice.private_key
wallet_alice_2 = Wallet(private_key_alice_str)

if wallet_alice_2.public_key == wallet_alice.public_key:
    print("Same Keys")
else:
    print("Error")


Same Keys


## 2. Transaktionen

Eine Transaktion besteht fundamental aus Inputs und Outputs. Da ein Input letztlich nichts anderes als eine Referenz auf einen zuvor erzeugten Output ist, genügt es ausschließlich die `TransactionOutput` Klasse zu definieren. (Die konkrete Gestaltung der `Transaction` Klasse wird später behandelt, nachdem die `TransactionOutput` Klasse beschrieben wurde.)

#### Transaction Output
Die grundlegenden Bausteine unserer Währung sind *Transaction Outputs*. Noch nicht ausgegegebene *Transaction Outputs* (kurz auch *UTXOs*) definieren die Geldmenge im System. D.h. die Summe aller *UTXOs* zu einem Zeitpunkt entspricht der gesamten Menge an verfügbaren *Fed Coins* (kurz FC) zu diesem Zeitpunkt. Ein Transaktionsinput ist einfach eine Referenz (per ID) auf einen noch nicht ausgegebenen *Transaction Output*, d.h. Input und Output können letztlich durch die selbe Klasse modelliert werden. Wir sprechen im Folgenden immer von *Transaction Output*, haben aber im gleichzeitig im Hinterkopf, dass ein Input letztlich nur eine Referenz auf einen *Transaction Output* ist. Ein *Transaction Output* soll folgenden Informationen beinhalten:

1. Zeitstempel
2. Besitzer/Empfänger
3. Wert
4. Id

Damit ein *Transaction Output* eindeutig ist (theoretisch könnte zweimal genau der gleiche Output zum gleichen Zeitpunkt generiert werden - praktisch aber sehr unwahrscheinlich, dass dieser Fall eintritt), fügen wir noch ein Feld mit einem Zufallswert (random) hinzu.

#### Hash eines TransactionOutputs

Zusatzinfo: Die genaue Funktionsweise des SH-Algorithmus, den wir zum hashen verdwenden, wird in diesem [video](https://www.youtube.com/watch?v=DMtFhACPnTY) erklärt.

Wir wollen, dass jeder *Transaction Output* eine eindeutige ID hat die an dessen Inhalt gekoppelt ist. D.h. wenn sich der Inhalt verändert, soll sich auch die ID verändern. Zusätzlich soll es unmöglich sein, durch die ID auf den Inhalt schließen zu können. Hierzu verwenden wir eine Hashfunktion. Die Hashfunktion benötigt als Input einen beliebig langen `String` und gibt einen Hash mit festgelegter Länge als Output zurück. Weil es nicht trivial ist ein generisches Pythonobjekt direkt in einen `String` umzuwandeln, gehen wir über den Umweg eines sogenannten OrderedDictionary's (`OrderedDict`). Aus jedem `OrderedDict` kann einfach ein String erzeugt werden (z.B. im `json` Format). D.h. um den Hash eines `TransactionOutput` Objekts zu erzeugen, müssen wir das Objekt
1. In ein `OrderedDict` umwandeln, 
2. Das `OrderedDict` in einen `String` umwandeln und 
3. Den Hashwert des `Strings` berechnen.  

Es ist sinnvoll die beiden Schritte 2 und 3 in eine Funktion auszulagern, weil wir die beiden Schritte in genau dieser Form für sämtliche Bestandteile unserer Kryptowährung (z.B. Blocks) widerverwenden können. Schritt 1 wird innerhalb der Klasse `TransactionOutput` implementiert, da sich der Schritt zwischen den einzelnen Objekttypen unserer Kryptowährung jeweils unterscheidet.

#### *Zusatzinfo: Defaultwerte der Parameter*
Die Parameter, die in der Implemetierung einer Methode mit Default-Werten initialisiert werden, müssen immer am Ende der Parameterliste stehen.

In [6]:
print(hash_stuff("Dominik Wunderlich"))
print(hash_stuff("Dominik J. Wunderlich"))

NameError: name 'hash_stuff' is not defined

In [7]:
def hash_stuff(ordered_dict_of_stuff):
    # Creates a SHA-256 hash of a the ordered_dict_of_stuff

    # Create a string representation of the ordered dict 
    # (via a json object)
    stuff_string = json.dumps(ordered_dict_of_stuff).encode('utf8')
    # Return the hexadecimal 256 bit hash of the string
    return hashlib.sha256(stuff_string).hexdigest()


class TransactionOutput:
    def __init__(self, recipient_public_key, value, random_val=None, id=None, timestamp=None):
        
        self.recipient = recipient_public_key  # This defines the owner of the coins
        self.value = value  # Simply the value of the output

        # Timestamp is none if new output is created, 
        # not none if old one is referenced/recreated
        if timestamp is None:
            self.time = time()
        else:
            self.time = timestamp

        # We need the random_val so that each output is guaranteed to be unique, even for same t.
        # I.e., in theory, if two TransactionOutputs (for the same recipient) 
        # are generated at the exact same time we still want to be able to distinguish them.
        if random_val is None:
            # self.random = 0
            self.random = random.randint(0, 10000000000000)
        else:
            # self.random = 0
            self.random = random_val

        # If it's a new transaction calculate the hash,
        # if it's an existing one read the hash from the argument given at initialization.
        if id is None:
            self.id = hash_stuff(self.odict_transaction_output())
        else:
            self.id = id

    # A hash function needs some String as input. 
    # To get a String from a TO object (not the entire one)
    # we use a OrderedDict representation which can be converted into a String.
    def odict_transaction_output(self):
        # TODO:
        # Return an OrderedDict with time, random, recipient, 
        # value (in that order)
        transaction_output_odict = OrderedDict({
            "time": self.time,
            "random": self.random,
            "recipient": self.recipient,
            "value": self.value
        })
        return transaction_output_odict

    def get_full_odict(self):
        # TODO:
        # Return an OrderedDict with time, random, recipient, 
        # value AND Id (in that order)
        response = self.odict_transaction_output()
        response["id"] = self.id
        return response


In [8]:
# Testing it:
# If you run this code multiple times, the output below should always change
# Try setting self.random = 0 in the constructor and check if it still does 
# (it should due to the timestamp)
transaction_output_to_alice = TransactionOutput(wallet_alice.public_key, 10)
print(transaction_output_to_alice.id)

46710000249f86b71b4435bf843d2db1d2a480415a978c3678a2727da3d9fcab


In [9]:
print(transaction_output_to_alice.get_full_odict())

OrderedDict([('time', 1541759881.9408698), ('random', 537161824421), ('recipient', '3059301306072a8648ce3d020106082a8648ce3d03010703420004c09adbceec8529afa226268ac405dd378d279d4a96566a39134ebd49e307054332961b1e33567435400aac564d0f73d5290acec333e261019aa0e8a95fa4e8c3'), ('value', 10), ('id', '46710000249f86b71b4435bf843d2db1d2a480415a978c3678a2727da3d9fcab')])


#### Transaction

Zusätzlich zu den fundamentalen Bestandteilen, den Inputs und Outputs, wird in einer Transaktion auch noch die Idendität des Senders und eine Signatur gespeichert. Damit eine Transaktion gültig ist, muss die Summe der Inputs gleich der Summe der Outputs sein. Das bedeutet, wenn Bob eine Banane von Alice für 5 FC kauft und dies mit einem Input von 10 FC bezahlen will, so muss er für die Gültigkeit einer Transaktion die restlichen 5 FC wieder an sich selbst zurücküberweisen. 

Zusätzlich wird für die Gültigkeit einer Transaktion gefordert, dass alle Inputs zum Zeitpunkt der Aufnahmen in die Blockchain im Besitz des Senders sind und die Signatur mit der Idendität des Senders übereinstimmt.

Da die Besitzverhältnisse der FC in der Blockchain erfasst werden, wird der die Besitzverhältnisse referenzierende Teil erst in der nächsten Veranstaltung ergänzt, da in dieser Veranstaltung dann auch die Blockchain eingeführt wird.


In [10]:
class Transaction:
    def __init__(self, sender_pub_key, inputs, outputs, timestamp=None, signature=None, transaction_id=None):
        # Note: This is somehow similar to the initialization
        # of TransactionOutput, except of the fact that there 
        # are different 'fields' in the Transaction class.
        self.sender = sender_pub_key
        self.inputs = inputs  # list of IDs referring to UTXOs
        self.outputs = outputs  # OrderedDict with id:TransactionOutput

        if timestamp is None:
            self.timestamp = time()  # include, to not have a hash collision for similar transactions
        else:
            self.timestamp = timestamp

        # Signature and ID are (re)created when the transaction is signed
        # Note: The coinbase transaction has no signature
        if signature is None:
            self.signature = ''
        else:
            self.signature = signature

        if transaction_id is None:
            self.id = self.hash_transaction()
        else:
            self.id = transaction_id

    def odict_transaction(self):
        # TODO:
        # Create an OrderedDict representation of the Transaction
        # We need this to sign it later on, therefore the 
        # signature is not included in the OrderedDict.
        # Order: timestamp,sender,inputs,outputs (odict itself)
        outputs_odict = OrderedDict()
        for output in self.outputs.values():
            outputs_odict[output.id] = output.get_full_odict()

        transaction_odict = OrderedDict({
            "timestamp": self.timestamp,
            "sender": self.sender,
            "inputs": self.inputs,
            "outputs": outputs_odict
        })

        return transaction_odict

    def hash_transaction(self):
        # Creates a hash of the transaction
        # We need this extra function since we have to
        # add the signature to the OrderedDict object
        # manually. Otherwise we could simply use the hash_stuff()
        # from above directly.

        transaction_dict = self.odict_transaction()
        transaction_dict['signature'] = self.signature
        transaction_string = json.dumps(transaction_dict).encode()
        
        return hashlib.sha256(transaction_string).hexdigest()

    def get_full_odict(self):
        # TODO:
        # Return a complete OrderedDict of the Transaction.
        # Order: timestamp,sender,inputs,outputs (odict itself)
        # signature, id
        
        response = self.odict_transaction()
        response["signature"] = self.signature
        response["id"] = self.id

        return response


In [11]:
# Testing the initialization part:
# Idea: send coins from Alice to Bob, i.e., Alice buys a coffee 
# from Bob for 5 FC.
# TODO:
# 1. Use the TO belonging to Alice from the 
# `transaction_output_to_alice` from above.
# 2. Create two TOs with each 5 FC, one belonging to Bob
# (price of the coffee) and the other one
# belonging to Alice (change). 
# 3. Create an OrderedDict of theses two TOs with id:TO,
# since this is the required input format for the transaction.
# 4. Create the transaction.

transaction_output_to_bob = TransactionOutput(wallet_bob.public_key, 5)
transaction_output_to_alice_2 = TransactionOutput(wallet_alice.public_key, 5)

outputs_od = OrderedDict({
    "transaction_output_to_bob": transaction_output_to_bob,
    "transaction_output_to_alice_2": transaction_output_to_alice_2
})

transaction_test = Transaction(wallet_alice.public_key, [transaction_output_to_alice.id], outputs_od)

In [12]:
%%add_to Transaction

def sign_transaction(self, private_key):
    # TODO:
    # Sign a transaction using the private_key and return the signature
    # 1. Import he key in the right format
    # 2. Create an instance oif the signer using the private key (DSS)
    # 3. Create the hash of  the message to sign
    # 4. Use the signer and the message to create the signature
    # 5. set self.signature
    # 6. Set the ID of the transaction after self.signature is set
    # 7. Return the signature
        
    pass
    
def verify_transaction_signature(self):
    # TODO:
    # Check if the transaction signature is valid,
    # i.e. does the signature match the Transaction object.
    # 1. Create a verifier instance using the public key of the sender
    # 2. Create the hash of the message
    # 3. Try to verify the signature using the verifier and the hash of
    # the message
    # 4. Return either True or False depending on the verification outcome
    # Note: In case the signature is not verified,
    # verifier.verify(SOMEHASH,SOMESIGNATURE) throws and error.
    
    pass

def verify_transaction_id(self):
    # Check if the id matched the Transaction content
     
    if self.id == self.hash_transaction():
        return True

    return False

In [13]:
# Testing  verify_transaction_signature() part (1):
# TODO:
# 1. Sign the Transaction transaction_alice_to_bob 
# using the right private_key (the one that coresponds
# the sender's public key).
# 2. Check if the id of the transactio changed after signing
# 3. Verify the signature




In [ ]:
# Testing the verify_transaction_signature() part (2):
# TODO:
# Use a wrong key to sign the transaction and check the result




In [ ]:
# Testing the verify_transaction_id() part (1):
# Sign it with the wrong signature 
# to get a wrong signature (for test reasons)
transaction_alice_to_bob.sign_transaction(wallet_bob.private_key)
id_with_wrong_signature = transaction_alice_to_bob.id

# Sign it with the right signature and test the id (actually trivial):
transaction_alice_to_bob.sign_transaction(wallet_alice.private_key)
print(transaction_alice_to_bob.verify_transaction_id())

In [ ]:
# Testing the verify_transaction_id() part (2):
# Now assign the wrong id and test again:
transaction_alice_to_bob.id = id_with_wrong_signature
print(transaction_alice_to_bob.verify_transaction_id())

## *Zusatz: Auslagerung abgeschlossener Programmteile*

Um den Überblick über den Code nicht zu verlieren, macht es in vielen Fällen Sinn, mehr oder weniger abgeschlossene Abschnitte des Codes in eigene \*.py Dateien auszulagern. Kleinere Änderungen am Code können auch in diesen Dateien durchgeführt werden. Um Programmcode zu testen, können wir auf die Klassen und Methoden in den ausgelagerten Dateien auch wieder über die jupyter Notebooks zugreifen. Bzgl. der Implementierung von FEDCoin gehen wir hierzu folgendermaßen vor:

1. Erstelle mit einem Texteditor deiner Wahl (Sublime wird empfohlen) eine Datei `FEDCoin.py` im selben Verzeichnis wie dieses Notebook und kopiere dort zuerst die Importe aus der obersten Zelle dieses Notebooks hinein und füge zusätzlich noch folgenden Import hinzu: `from utils import *`.  Kopiere dann die jeweils fotgeschrittensten Implementierungen der Klassen `Wallet`, `TransactionOutput` und `Transaction` in diese Datei hinein.

2. Erstelle eine zweite Datei mit dem Namen `utils.py` die zunächst nur die Funktion `hash_shuff()` und die für die Funktion notwendigen Importe (`hashlib`, `json`) beinhaltet.

3. Importiere den Inhalt beider Dateien in ein jupyter Notebook und erzeuge z.B. ein neues Wallet damit. Um sämtliche Inhalte aus einer \*.py Datei zu importieren, verwendet man einen Befehl à la `from FEDCoin import *` (siehe Beispiel unten).


In [ ]:
# Testing the import of code from *.py files in the same folder:
# Note: To make sure that you don't use the cached classes
# go to the menue bar of this notebook and select
# --->Kernel--->Restart & Clear Output
# Then run the code in this cell.

from FEDCoin import *
from utils import *

wallet_peter = Wallet()
print(wallet_peter.public_key)
print("\n")

test_od = OrderedDict({'bla':'blub'})
print(hash_stuff(test_od))